In [ ]:
import os, sys
from io import StringIO

import pandas as pd
from datetime import datetime, timezone, timedelta
import psycopg2
import time

from matplotlib import pyplot as plt

import ccxt

In [2]:
timeframe = '1m'

start_str_dt = '2025-02-01T00:00:00Z'
end_str_dt = '2025-05-01T00:00:00Z'

limit = 1000  # Максимум для binance = 1000
counter = 0

parquet_path = '03_05_1m.parquet'

# Формируем исторические данные с пампами

In [21]:
tickers = [
    # 'SOL/USDT', 
    # 'XRP/USDT', 
    # 'TON/USDT', 
    # 'ADA/USDT', 
    # 'MNT/USDT'
    'NEAR/USDT',
    'DOGE/USDT'
]

exchange = ccxt.binance()
dfs = []

for idx, ticker in enumerate(tickers, 1):
    since = exchange.parse8601(start_str_dt)
    end = exchange.parse8601(end_str_dt)
    all_bars = []

    while since < end:
        try:
            bars = exchange.fetch_ohlcv(ticker, timeframe=timeframe, since=since, limit=limit)
            if not bars:
                break
            all_bars.extend(bars)
            since = bars[-1][0] + 60_000
            time.sleep(exchange.rateLimit / 1000)
        except ccxt.BaseError as e:
            print(e)
            time.sleep(10)

    if all_bars:
        df = pd.DataFrame(all_bars, columns=['timestamp','open','high','low','close','volume'])
        df['datetime'] = (
            pd.to_datetime(df.pop('timestamp'), unit='ms', utc=True)
            .dt.tz_localize(None)
        )
        df['ticker'] = ticker
        dfs.append(df[['ticker','datetime','open','high','low','close','volume']])
        print(f"{idx}. {ticker}: {dfs[-1].shape}")
    else:
        print(f"{idx}. {ticker}: no data fetched")

1. NEAR/USDT: (129000, 7)
2. DOGE/USDT: (129000, 7)


In [3]:
df_all = pd.concat(dfs, ignore_index=True)
parquet_path = '03_05_1m.parquet'

if os.path.exists(parquet_path):
    # Если файл существует, читаем старые данные и объединяем
    old_df = pd.read_parquet(parquet_path)
    df_all = pd.concat([old_df, df_all], ignore_index=True)
    # Удаляем дубликаты по уникальным столбцам (ticker, datetime)
    df_all = df_all.drop_duplicates(subset=['ticker', 'datetime'])

df_all.to_parquet(parquet_path)

# Чтение

In [ ]:
df = pd.read_parquet(parquet_path)
df = df.loc[(df.ticker == 'ADA/USDT') & (df.datetime.between('2025-03-02 00:00:00', '2025-03-02 23:59:59'))]

In [5]:
import plotly.graph_objs as go

fig = go.Figure(
    data=[go.Candlestick(
        x=df['datetime'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='OHLC'
    )]
)

fig.update_layout(
    title='BTC/USDT Свечной график',
    xaxis_title='Время',
    yaxis_title='Цена (USDT)'
)

fig.show()